In [16]:
import os
from dotenv import load_dotenv


load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [17]:
from crewai import Crew, Task

In [18]:
from crewai_tools import tool

import yfinance as yf


class Tools:

    @tool("One month stock price history")
    def stock_price(ticker):
        """
        Useful to get a month's worth of stock price data as CSV.
        The input of this tool should a ticker, for example AAPL, NET, TSLA etc...
        """
        stock = yf.Ticker(ticker)
        return stock.history(period="1mo").to_csv()

    @tool("Stock news URLs")
    def stock_news(ticker):
        """
        Useful to get URLs of news articles related to a stock.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return list(map(lambda x: x["link"], stock.news))

    @tool("Company's income statement")
    def income_stmt(ticker):
        """
        Useful to get the income statement of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.income_stmt.to_csv()

    @tool("Balance sheet")
    def balance_sheet(ticker):
        """
        Useful to get the balance sheet of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.balance_sheet.to_csv()

    @tool("Get insider transactions")
    def insider_transactions(ticker):
        """
        Useful to get insider transactions of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.insider_transactions.to_csv()

In [19]:
from crewai import Agent


from crewai_tools import ScrapeWebsiteTool


class Agents:

    def technical_analyst(self):
        return Agent(
            role="Technical Analyst",
            goal="Analyses the movements of a stock and provides insights on trends, entry points, resistance and support levels.",
            backstory="An expert in technical analysis, you're known for your ability to predict stock movements and trends based on historical data. You provide valuable insights to your customers.",
            verbose=True,
            tools=[
                Tools.stock_price,
            ],
        )

    def researcher(self):
        return Agent(
            role="Researcher",
            goal="Gathers, interprets and summarizes vasts amounts of data to provide a comprehensive overview of the sentiment and news surrounding a stock.",
            backstory="You're skilled in gathering and interpreting data from various sources to give a complete picture of a stock's sentiment and news. You read each data source carefuly and extract the most important information. Your insights are crucial for making informed investment decisions.",
            verbose=True,
            tools=[
                Tools.stock_news,
                ScrapeWebsiteTool(),
            ],
        )

    def financial_analyst(self):
        return Agent(
            role="Financial Analyst",
            goal="Uses financial statements, insider trading data, and other financial metrics to evaluate a stock's financial health and performance.",
            backstory="You're a very experienced investment advisor who uses a combination of technical and fundamental analysis to provide strategic investment advice to your clients. You look at a company's financial health, market sentiment, and qualitative data to make informed recommendations.",
            verbose=True,
            tools=[
                Tools.balance_sheet,
                Tools.income_stmt,
                Tools.insider_transactions,
            ],
        )

    def hedge_fund_manager(self):
        return Agent(
            role="Hedge Fund Manager",
            goal="Manages a portfolio of stocks and makes strategic investment decisions to maximize returns using insights from financial analysts, technical analysts, and researchers.",
            backstory="You're a seasoned hedge fund manager with a proven track record of making profitable investment decisions. You're known for your ability to manage risk and maximize returns for your clients.",
            verbose=True,
        )

In [20]:
from crewai import Task


class Tasks:

    def research(self, agent):
        return Task(
            description="Gather and analyze the latest news and market sentiment surrounding the stock of {company}. Provide a summary of the news and any notable shifts in market sentiment.",
            expected_output=f"Your final answer MUST be a detailed summary of the news and market sentiment surrounding the stock. Include any notable shifts in market sentiment and provide insights on how these factors could impact the stock's performance.",
            agent=agent,
            output_file="stock_news.md",
        )

    def technical_analysis(self, agent):
        return Task(
            description="Conduct a detailed technical analysis of the price movements of {company}'s stock and trends identify key support and resistance levels, chart patterns, and other technical indicators that could influence the stock's future performance. Use historical price data and technical analysis tools to provide insights on potential entry points and price targets.",
            expected_output=f"Your final answer MUST be a detailed technical analysis report that includes key support and resistance levels, chart patterns, and technical indicators. Provide insights on potential entry points, price targets, and any other relevant information that could help your customer make informed investment decisions.",
            agent=agent,
            output_file="technical_analysis.md",
        )

    def finacial_analysis(self, agent):
        return Task(
            description="Analyze {company}'s financial statements, insider trading data, and other financial metrics to evaluate the stock's financial health and performance. Provide insights on the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.",
            expected_output=f"Your final answer MUST be a detailed financial analysis report that includes insights on the company's financial health, performance, and valuation. Provide an overview of the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.",
            agent=agent,
            output_file="financial_analysis.md",
        )

    def investment_recommendation(self, agent, context):
        return Task(
            description="Based on the research, technical analysis, and financial analysis reports, provide a detailed investment recommendation for {company}'s stock. Include your analysis of the stock's potential risks and rewards, and provide a clear rationale for your recommendation.",
            expected_output=f"Your final answer MUST be a detailed investment recommendation report to BUY or SELL the stock that includes your analysis of the stock's potential risks and rewards. Provide a clear rationale for your recommendation based on the research, technical analysis, and financial analysis reports.",
            agent=agent,
            context=context,
            output_file="investment_recommendation.md",
        )

In [21]:
from crewai import Crew
from crewai.process import Process
from langchain_openai import ChatOpenAI

agents = Agents()
tasks = Tasks()

researcher = agents.researcher()
technical_analyst = agents.technical_analyst()
financial_analyst = agents.financial_analyst()
hedge_fund_manager = agents.hedge_fund_manager()

research_task = tasks.research(researcher)
technical_task = tasks.technical_analysis(technical_analyst)
financial_task = tasks.finacial_analysis(financial_analyst)
recommend_task = tasks.investment_recommendation(
    hedge_fund_manager,
    [
        research_task,
        technical_task,
        financial_task,
    ],
)

crew = Crew(
    agents=[
        researcher,
        technical_analyst,
        financial_analyst,
        hedge_fund_manager,
    ],
    tasks=[
        research_task,
        technical_task,
        financial_task,
        recommend_task,
    ],
    verbose=2,
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model="gpt-4o-mini"),
    memory=True,
)

result = crew.kickoff(
    inputs=dict(
        company="Salesforce",
    ),
)

2024-10-28 16:46:27,614 - 8440214080 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Gather and analyze the latest news and market sentiment surrounding the stock of Salesforce. Provide a summary of the news and any notable shifts in market sentiment.


> Entering new CrewAgentExecutor chain...
I need to gather the latest news and market sentiment regarding Salesforce's stock. This involves understanding recent developments, news articles, analyst opinions, and any significant shifts in investor sentiment that could affect the stock's performance. To accomplish this, I will delegate the task of researching the news and market sentiment to the Researcher, as they are equipped to gather comprehensive information from various sources.

Action: Delegate work to co-worker  
Action Input: {"task": "Gather and analyze the latest news and market sentiment surrounding the stock of Salesforce. Provide a detailed summary of the news and any notable shifts in market sentiment, including insights on how these factors cou

2024-10-28 16:47:30,487 - 8440214080 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--8695e4addd45cee8bad1d2663eb2c66f622c9a3839ddb0a828ceea9ff2318702


Thought: I have gathered the balance sheet and income statement data for Salesforce (CRM). Now, I need to retrieve the insider trading data to analyze any trends in insider transactions that may provide additional insights into the company's financial health and performance.

Action: Get insider transactions  
Action Input: {"ticker": "CRM"}   

I tried reusing the same input, I must stop using this action input. I'll try something else instead.





2024-10-28 16:47:32,469 - 8440214080 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--8695e4addd45cee8bad1d2663eb2c66f622c9a3839ddb0a828ceea9ff2318702


...

Thought: I now need to gather insider trading data for Salesforce Inc. to evaluate any potential concerns regarding insider transactions.

Action: Get insider transactions  
Action Input: {"ticker": "CRM"}  
 

,Shares,Value,URL,Text,Insider,Position,Transaction,Start Date,Ownership
0,897,255904.0,,Sale at price 285.29 per share.,HARRIS G PARKER,Officer and Director,,2024-10-23,D
1,889,253622.0,,Sale at price 285.29 per share.,SCHMAIER ROBERT DAVID,President,,2024-10-23,D
2,979,279298.0,,Sale at price 285.29 per share.,MILLHAM BRIAN,President,,2024-10-23,D
3,413,117824.0,,Sale at price 285.29 per share.,REDDY SUNDEEP G,Officer,,2024-10-23,D
4,897,255904.0,,Sale at price 285.29 per share.,TALLAPRAGADA SRINIVAS,President,,2024-10-23,D
5,897,255904.0,,Sale at price 285.29 per share.,WEAVER AMY E,President,,2024-10-23,D
6,6161,1273656.0,,Conversion of Exercise of derivative security at price 186.51 - 218.21 per share.,MILLHAM BRIAN,President,,2024-10-22,D
7,1948,,,,MILLHAM BRIAN,President,

2024-10-28 16:49:22,078 - 8440214080 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--201d153ca0e1a17048a862dcf37aecdcbe6955d18f95feba3f2cf44263196c51


Thought: I need to gather detailed financial information about Salesforce Inc. to conduct a comprehensive analysis, including its balance sheet, income statement, and insider trading data.

Action: Balance sheet  
Action Input: {"ticker": "CRM"}  
 

,2024-01-31,2023-01-31,2022-01-31,2021-01-31,2020-01-31
Treasury Shares Number,64000000.0,28000000.0,,,
Ordinary Shares Number,971000000.0,981000000.0,989000000.0,919000000.0,
Share Issued,1035000000.0,1009000000.0,989000000.0,919000000.0,
Net Debt,954000000.0,3585000000.0,5132000000.0,,
Total Debt,12588000000.0,14088000000.0,13985000000.0,6281000000.0,
Tangible Book Value,5748000000.0,2666000000.0,1216000000.0,11061000000.0,
Invested Capital,69072000000.0,68960000000.0,68727000000.0,44166000000.0,
Working Capital,2443000000.0,504000000.0,1062000000.0,4161000000.0,
Net Tangible Assets,5748000000.0,2666000000.0,1216000000.0,11061000000.0,
Capital Lease Obligations,3162000000.0,3487000000.0,3389000000.0,3608000000.0,
Common Stock Equity,5964

2024-10-28 16:49:24,478 - 8440214080 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--309d8ddbe9e2b4bca17ae9a9576b0d625a859b750c7626eb290a696cec2ba503


Thought: I now need to gather the income statement for Salesforce Inc. to assess the company's revenue and earnings performance.

Action: Company's income statement  
Action Input: {"ticker": "CRM"}  
 

,2024-01-31,2023-01-31,2022-01-31,2021-01-31,2020-01-31
Tax Effect Of Unusual Items,-202400000.0,-224070000.0,69561357.70235,585900000.0,
Tax Rate For Calcs,0.16,0.21,0.057441,0.27,
Normalized EBITDA,11223000000.0,6711000000.0,2635000000.0,1131000000.0,
Total Unusual Items,-1265000000.0,-1067000000.0,1211000000.0,2170000000.0,
Total Unusual Items Excluding Goodwill,-1265000000.0,-1067000000.0,1211000000.0,2170000000.0,
Net Income From Continuing Operation Net Minority Interest,4136000000.0,208000000.0,1444000000.0,4072000000.0,
Reconciled Depreciation,3959000000.0,3786000000.0,3298000000.0,2846000000.0,
Reconciled Cost Of Revenue,8541000000.0,8360000000.0,7026000000.0,5438000000.0,
EBITDA,9958000000.0,5644000000.0,3846000000.0,3301000000.0,
EBIT,5999000000.0,1858000000.0,548000000.0,45